# Adding new samples on the main table

The way to extrat procurement reports data from SECOP I is requesting to API Socrata. So we could use several queries to get different sets of reports.

The query that we were working bring us the reports of third part paths, if we run the same code we got 857 records

In [1]:
# Parameters
SECOPI_PROCESS_API = 'f789-7hwg'
SECOPII_PROCESS_API = 'jbjy-vk9h'
SECOPII_ADDITIONS_API = 'cb9c-h8sn'

QUERIES_FOLDER = 'bin\queries'

In [2]:
# General functions
import os
import pandas as pd
import numpy as np
import ETL as etl

# Get query files function
def get_path(folder, file_path):
    current_directory = os.getcwd()
    return os.path.join(current_directory, '..', folder, file_path)


def get_query(folder, file_path):
    path = get_path(folder, file_path)
    # try to get the query
    with open(path, "r", encoding="utf8") as query_file:
        query = query_file.read()
    
    return query


In [3]:
# Get vias rurales procurements
query = get_query(QUERIES_FOLDER, 'request_data_vias_rurales_v2.sql')
vias_rurales_data = etl.extract_data(query, id_data=SECOPI_PROCESS_API, api_key=None)

# apply the cleaning process
vias_rurales_data = etl.process_data(vias_rurales_data)

El numero de contratos extraidos: 3856


## Getting samples from several escenarios
So using the same process above, we are going to get more records based on different escenarios as show here:
* "Vias secundarias" projects
* Extend the timeline until end of 2023
* Record from SECOP II
* Other method to catch the interested records

### "Vias secundarias" projects
We ran a query with new set of conditions to get these kind of records. As result we got 41 record extra
Note a new process were added to remove the overlaped records between the first dataset and this. So it found 580 record overlaped

In [22]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_secundarias_v2.sql')

vias_secundarias_data = etl.extract_data(query, id_data=SECOPI_PROCESS_API, api_key=None)

# apply the cleaning process
vias_secundarias_data = etl.process_data(vias_secundarias_data)

# remove the records those is alredy in the first data set
unique_contracts_vr = list(set(vias_rurales_data['CONTRACT_ID']))

unique_contracts_vs = [x for x in list(set(vias_secundarias_data['CONTRACT_ID'])) 
                        if not(x in unique_contracts_vr)]

vias_secundarias_data = vias_secundarias_data[vias_secundarias_data["CONTRACT_ID"].isin(unique_contracts_vs)]

print("Numero de contratos luego de remover overlap: {records}".format(records = len(vias_secundarias_data)))

El numero de contratos extraidos: 2545
Numero de contratos luego de remover overlap: 1857


## Extend the timeline until end of 2023
Until now the queries have a condition to get records before to jan-2021. So if we extend the timeline to jan-2024 90 records are founded about "vias rurales" and 7 to "vias secundarias"

In [23]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_rurales_t2_v2.sql')

vias_ruralest2_data = etl.extract_data(query, id_data=SECOPI_PROCESS_API, api_key=None)

# apply the cleaning process
vias_ruralest2_data = etl.process_data(vias_ruralest2_data)

# remove the records those is alredy in the first data set
unique_contracts = unique_contracts_vr + unique_contracts_vs

unique_contracts_vrt2 = [x for x in list(set(vias_ruralest2_data['CONTRACT_ID'])) 
                        if not(x in unique_contracts)]

vias_ruralest2_data = vias_ruralest2_data[vias_ruralest2_data["CONTRACT_ID"].isin(unique_contracts_vrt2)]

print("Numero de contratos luego de remover overlap: {records}".format(records = len(vias_ruralest2_data)))

El numero de contratos extraidos: 472
Numero de contratos luego de remover overlap: 472


In [24]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_secundarias_t2_v2.sql')

vias_secundariast2_data = etl.extract_data(query, id_data=SECOPI_PROCESS_API, api_key=None)

# apply the cleaning process
vias_secundariast2_data = etl.process_data(vias_secundariast2_data)

# remove the records those is alredy in the first data set
unique_contracts = unique_contracts + unique_contracts_vrt2

unique_contracts_vst2 = [x for x in list(set(vias_secundariast2_data['CONTRACT_ID'])) 
                        if not(x in unique_contracts)]

vias_secundariast2_data = vias_secundariast2_data[vias_secundariast2_data["CONTRACT_ID"].isin(unique_contracts_vst2)]

print("Numero de contratos luego de remover overlap: {records}".format(records = len(vias_secundariast2_data)))

El numero de contratos extraidos: 174
Numero de contratos luego de remover overlap: 120


## Record from SECOP II

In [25]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_rurales_sii_v2.sql')

vias_ruralessii_data = etl.extract_data(query, id_data=SECOPII_PROCESS_API, api_key=None)

# apply the cleaning process
vias_ruralessii_data = etl.process_data(vias_ruralessii_data)

El numero de contratos extraidos: 23


In [26]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_rurales_sii_t2_v2.sql')

vias_ruralessiit2_data = etl.extract_data(query, id_data=SECOPII_PROCESS_API, api_key=None)

# apply the cleaning process
vias_ruralessiit2_data = etl.process_data(vias_ruralessiit2_data)

El numero de contratos extraidos: 122


In [27]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_secundarias_sii_v2.sql')

vias_secundariassii_data = etl.extract_data(query, id_data=SECOPII_PROCESS_API, api_key=None)

if len(vias_secundariassii_data) > 0:

    # apply the cleaning process
    vias_secundariassii_data = etl.process_data(vias_secundariassii_data)

    # remove the records those is alredy in the first data set
    unique_contracts = list(set(vias_ruralessii_data['CONTRACT_ID'])) + list(set(vias_ruralessiit2_data['CONTRACT_ID']))

    unique_contracts_siivs = [x for x in list(set(vias_secundariassii_data['CONTRACT_ID'])) 
                                    if not(x in unique_contracts)]

    vias_secundariassii_data = vias_secundariassii_data[vias_secundariassii_data["CONTRACT_ID"].isin(unique_contracts_siivs)]

    print("Numero de contratos luego de remover overlap: {records}".format(records = len(vias_secundariassii_data)))

El numero de contratos extraidos: 29
Numero de contratos luego de remover overlap: 17


In [28]:
query = get_query(QUERIES_FOLDER, 'request_data_vias_secundarias_sii_t2_v2.sql')

vias_secundariassiit2_data = etl.extract_data(query, id_data=SECOPII_PROCESS_API, api_key=None)

if len(vias_secundariassiit2_data) > 0:
    # apply the cleaning process
    vias_secundariassiit2_data = etl.process_data(vias_secundariassiit2_data)

    # remove the records those is alredy in the first data set
    unique_contracts_siivst2 = [x for x in list(set(vias_secundariassiit2_data['CONTRACT_ID'])) 
                                        if not(x in unique_contracts)]

    vias_secundariassiit2_data = vias_secundariassiit2_data[vias_secundariassiit2_data["CONTRACT_ID"].isin(unique_contracts_siivst2)]

    print("Numero de contratos luego de remover overlap: {records}".format(records = len(vias_secundariassiit2_data)))

El numero de contratos extraidos: 79
Numero de contratos luego de remover overlap: 34


## Summary


In [29]:
vias_rurales_data['GROUP'] = 'Vias Rurales hasta 2020'
#vias_secundarias_data['GROUP'] = 'Vias secundarias hasta 2020'
vias_ruralest2_data['GROUP'] = 'Vias Rurales 2021-2023'
#vias_secundariast2_data['GROUP'] = 'Vias secundarias 2021-2023'
vias_ruralessii_data['GROUP'] = 'Vias Rurales en SECOP II hasta 2020'
vias_ruralessiit2_data['GROUP'] = 'Vias Rurales en SECOP II 2021-2023'
#vias_secundariassii_data['GROUP'] = 'Vias secundarias en SECOP II hasta 2020'
#vias_secundariassiit2_data['GROUP'] = 'Vias secundarias en SECOP II 2021-2023'

# collected_data = pd.concat([vias_rurales_data, vias_secundarias_data, 
#                             vias_ruralest2_data, vias_secundariast2_data
#                            , vias_ruralessii_data, vias_ruralessiit2_data,
#                            vias_secundariassii_data, vias_secundariassiit2_data
#                            ])

collected_data = pd.concat([vias_rurales_data, vias_ruralest2_data,
                            vias_ruralessii_data, vias_ruralessiit2_data
                           ])


In [30]:
import numpy as np
percentile = [25,50,75]
thresholds = np.percentile(collected_data['FINAL_COST_NORM'], percentile)
thresholds = [0] + list(thresholds) + [np.max(collected_data['FINAL_COST_NORM']) + 1]
categories = np.digitize(collected_data['FINAL_COST_NORM'], thresholds)
collected_data['COST_GROUP_QUANTILE'] = categories

In [31]:

collected_data.to_csv(get_path('data', 'collected_obra_data_excel_v2.csv'), index=False, sep=';', decimal=',')
collected_data.to_csv(get_path('data', 'collected_obra_data_v2.csv'), index=False)

def summary(x):
    data = {}

    data['unique contracts'] = x['CONTRACT_ID'].size
    data['value total'] = x['CONTRACT_VALUE_NORM'].sum()
    data['value average'] = x['CONTRACT_VALUE_NORM'].mean()
    data['cost deviation total'] = x['COST_DEVIATION_NORM'].sum()
    data['cost deviation average'] = x['COST_DEVIATION_NORM'].mean()

    return pd.Series(data)

collected_data.groupby('GROUP').apply(summary)

,unique contracts,value total,value average,cost deviation total,cost deviation average
GROUP,,,,,
Vias Rurales 2021-2023,472.0,3.922339e+05,831.004036,17.974420,0.038081
Vias Rurales en SECOP II 2021-2023,122.0,1.693699e+05,1388.278110,0.000000,0.000000
Vias Rurales en SECOP II hasta 2020,23.0,2.389013e+05,10387.015183,0.000000,0.000000
Vias Rurales hasta 2020,3856.0,2.528868e+06,655.826702,137.090383,0.035552
